In [1]:
import pandas as pd
import numpy as np

In [6]:
import glob, os

fake_list = glob.glob("dataset/fake/*.txt")
true_list = glob.glob("dataset/true/*.txt")


In [20]:
df = pd.DataFrame(columns=['file_num','text','label'])

# adding all labeled news
i = 1
for file_list in [fake_list, true_list]:
    for file in file_list:
        fileName = os.path.basename(os.path.splitext(file)[0]) # splitext get name without extension on position 0 and basename just the filename

        with open(file, 'r', encoding="utf8") as file_open:
            data = file_open.read().replace('\n', '')

        if i == 1:
            label = "fake"
        else:
            label = "true"
            
        df = df.append({
         "file_num": fileName,
         "text":  data,
         "label": label
        }, ignore_index=True)
        
    i+=1

df.head()

,file_num,text,label
0,1,Kátia Abreu diz que vai colocar sua expulsão e...,fake
1,10,"Dr. Ray peita Bolsonaro, chama-o de conservad...",fake
2,100,Reinaldo Azevedo desmascarado pela Polícia Fed...,fake
3,1000,Relatório assustador do BNDES mostra dinheiro ...,fake
4,1001,"Radialista americano fala sobre o PT: ""Eles ve...",fake


In [23]:
df['label'].value_counts()

true    3600
fake    3600
Name: label, dtype: int64